## Voy a hacer un analisis de la relacion entre que palabras contienen los tweets y su veracidad. Este analisis abarca, cantidad de palabras, cantidad de caracteres y palabras que aparecen.

#### Importo las librerias necesarias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### Creo el data frame en base al csv

In [2]:
tweets = pd.read_csv('csv/train.csv', encoding='latin-1')

#### Analizo como esta formado mi data frame

In [3]:
tweets.sample(n=3)

,id,keyword,location,text,target
5536,7897,radiation%20emergency,NaN,I liked a @YouTube video http://t.co/V57NUgmGK...,1
232,331,annihilated,"West Lancashire, UK.",@stormbeard @steel_lord I seen Judas Priest in...,0
5957,8508,screaming,"Moore, OK",@noahshack he's hot &amp; he can sing I'm scre...,0


#### Tanto la columna de location, keyword e id son irrelevantes para este analisis.

In [4]:
del(tweets['keyword'])
del(tweets['location'])
tweets.sample(n=3)

,id,text,target
4900,6975,Tunisia beach massacre linked to March terror ...,1
4778,6799,kabwandi_: Breaking news! Unconfirmed! I just ...,0
1892,2723,A diamond is just a piece of charcoal that han...,1


#### me guardo cuantos caracteres contiene cada tweet

In [5]:
tweets['nºcaracteres'] = tweets['text'].str.len()

In [6]:
tweets.sample(n=3)

,id,text,target,nºcaracteres
4729,6725,Deal of The Day : http://t.co/US0qQqhQVj Brand...,0,141
7426,10623,Police Officer Wounded Suspect Dead After Exch...,1,58
7075,10136,#Tigers Wonder how much the upheaval with team...,0,86


#### Ahora me interesa separar el texto en palabras.

In [7]:
tweets['palabras']=tweets['text'].str.split()

In [8]:
tweets.head()

,id,text,target,nºcaracteres,palabras
0,1,Our Deeds are the Reason of this #earthquake M...,1,69,"[Our, Deeds, are, the, Reason, of, this, #eart..."
1,4,Forest fire near La Ronge Sask. Canada,1,38,"[Forest, fire, near, La, Ronge, Sask., Canada]"
2,5,All residents asked to 'shelter in place' are ...,1,133,"[All, residents, asked, to, 'shelter, in, plac..."
3,6,"13,000 people receive #wildfires evacuation or...",1,65,"[13,000, people, receive, #wildfires, evacuati..."
4,7,Just got sent this photo from Ruby #Alaska as ...,1,88,"[Just, got, sent, this, photo, from, Ruby, #Al..."


#### Reordeno las columnas para mayor claridad.

In [9]:
tweets = tweets[['id','text','palabras','nºcaracteres','target']]

In [10]:
tweets.head()

,id,text,palabras,nºcaracteres,target
0,1,Our Deeds are the Reason of this #earthquake M...,"[Our, Deeds, are, the, Reason, of, this, #eart...",69,1
1,4,Forest fire near La Ronge Sask. Canada,"[Forest, fire, near, La, Ronge, Sask., Canada]",38,1
2,5,All residents asked to 'shelter in place' are ...,"[All, residents, asked, to, 'shelter, in, plac...",133,1
3,6,"13,000 people receive #wildfires evacuation or...","[13,000, people, receive, #wildfires, evacuati...",65,1
4,7,Just got sent this photo from Ruby #Alaska as ...,"[Just, got, sent, this, photo, from, Ruby, #Al...",88,1


#### Me interesa saber cuantas palabras tiene que cada tweet.

In [11]:
type(tweets['palabras'])

pandas.core.series.Series

#### Guardo la cantidad de palabras en una nueva columna

In [12]:
lista_auxiliar = []
for i in tweets['palabras']:
    lista_auxiliar.append( len(i) )

In [13]:
tweets['nºpalabras'] = lista_auxiliar

In [14]:
tweets.head()

,id,text,palabras,nºcaracteres,target,nºpalabras
0,1,Our Deeds are the Reason of this #earthquake M...,"[Our, Deeds, are, the, Reason, of, this, #eart...",69,1,13
1,4,Forest fire near La Ronge Sask. Canada,"[Forest, fire, near, La, Ronge, Sask., Canada]",38,1,7
2,5,All residents asked to 'shelter in place' are ...,"[All, residents, asked, to, 'shelter, in, plac...",133,1,22
3,6,"13,000 people receive #wildfires evacuation or...","[13,000, people, receive, #wildfires, evacuati...",65,1,8
4,7,Just got sent this photo from Ruby #Alaska as ...,"[Just, got, sent, this, photo, from, Ruby, #Al...",88,1,16


In [15]:
#### Guardo en cada tweet la cantidad de 

#### Reordeno

In [16]:
tweets = tweets[['id','text','palabras','nºcaracteres','nºpalabras','target']]

In [17]:
tweets.head()

,id,text,palabras,nºcaracteres,nºpalabras,target
0,1,Our Deeds are the Reason of this #earthquake M...,"[Our, Deeds, are, the, Reason, of, this, #eart...",69,13,1
1,4,Forest fire near La Ronge Sask. Canada,"[Forest, fire, near, La, Ronge, Sask., Canada]",38,7,1
2,5,All residents asked to 'shelter in place' are ...,"[All, residents, asked, to, 'shelter, in, plac...",133,22,1
3,6,"13,000 people receive #wildfires evacuation or...","[13,000, people, receive, #wildfires, evacuati...",65,8,1
4,7,Just got sent this photo from Ruby #Alaska as ...,"[Just, got, sent, this, photo, from, Ruby, #Al...",88,16,1


#### Ya tengo bastante ordenado el data frame. Procedo a analizar.

## Palabras que tienen mas ocurrencias en tweets.

#### Procedo a crear un nuevo dataframe, donde levanto cada palabra junto con su target y id de tweet.

In [18]:
palabras = tweets.explode('palabras')
del(palabras['text'])
del(palabras['nºcaracteres'])
del(palabras['nºpalabras'])
palabras.rename({'palabras': 'palabra', 'id': 'id_tweet','target': 'target_tweet' }, axis=1, inplace=True)
palabras.reset_index(inplace = True, drop = True)
palabras.head()

,id_tweet,palabra,target_tweet
0,1,Our,1
1,1,Deeds,1
2,1,are,1
3,1,the,1
4,1,Reason,1


#### En el analisis no voy a discrimnar por mayusculas, paso todo a minuscula.

In [19]:
palabras['palabra'] = palabras['palabra'].str.lower()
palabras.head()

,id_tweet,palabra,target_tweet
0,1,our,1
1,1,deeds,1
2,1,are,1
3,1,the,1
4,1,reason,1


#### Hay casos donde la palabra empieza con un caracter especial, ejemplo de #.

In [20]:
palabras.iloc[7]

id_tweet                  1
palabra         #earthquake
target_tweet              1
Name: 7, dtype: object

#### Yo quiero eliminar dichos caracteres, ya que en el caso que se encuentren ""#earthquake" y "earthquake" busco que cuenten como la misma palabra. 

In [21]:
import re
palabras_limpias = []
for palabra in palabras['palabra']:
    palabra_limpia = re.sub('[^A-Za-z0-9]+','', palabra)
    palabras_limpias.append(palabra_limpia)

In [22]:
palabras['palabra']=palabras_limpias

In [23]:
palabras.iloc[7]

id_tweet                 1
palabra         earthquake
target_tweet             1
Name: 7, dtype: object

#### Ahora ya borramos los caracteres especiales, el siguiente paso es borrar las palabras que aparecen 2 veces en un tweet. Esto es para que las ocurrencias no cuenten doble.


In [24]:
palabras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113461 entries, 0 to 113460
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id_tweet      113461 non-null  int64 
 1   palabra       113461 non-null  object
 2   target_tweet  113461 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 2.6+ MB


In [25]:
palabras.drop_duplicates(subset=['palabra','id_tweet'], keep="first", inplace = True)
palabras.reset_index(inplace = True, drop = True)

In [26]:
palabras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107617 entries, 0 to 107616
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id_tweet      107617 non-null  int64 
 1   palabra       107617 non-null  object
 2   target_tweet  107617 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 2.5+ MB


#### Ya fueron eliminadas las palabras repetidas de cada tweet, todo listo para el analisis

In [30]:
agrupadas=palabras.groupby(['palabra']).agg({'target_tweet':['mean','count']})
agrupadas.sample(3)

target_tweet      
                          mean count
palabra                             
httptcoipxpi3me16          0.0     1
childish                   1.0     1
starring                   0.4     5

#### Considero que las palabras que aparecen menos del 0.2% de los tweets son irrelevantes para este analisis. Suelen ser 'one hit'

In [36]:
minimo_de_ocurrencias = (0.2/100)*len(tweets.index)
minimo_de_ocurrencias

15.226

In [37]:
agrupadas = agrupadas.loc[agrupadas[('target_tweet','count')]>minimo_de_ocurrencias,:]

In [41]:
agrupadas.sample(3)

target_tweet      
                mean count
palabra                   
severe      1.000000    44
white       0.351351    37
trauma      0.484848    33

#### Aplano el level de las columnas y cambio el nombre de las mismas.

In [ ]:
level 0 = 
level 1 = 